In [1]:
import sys
import os
import importlib
sys.path.insert(0, os.path.abspath("../data_model/"))

In [ ]:
import pandas as pd
import numpy as np
from pydantic import ValidationError
import data_model
import enums as e
from utils import extract_base_type, add_enum_label_columns, add_list_objects, add_synthetic_records

In [3]:
importlib.reload(data_model)
importlib.reload(e)
from data_model import Respondent, Employee, AirPassenger, Trip

In [ ]:
external_dir = "../data/external"
interim_dir = "../data/interim"
processed_dir = "../data/processed"

input_file = os.path.join(external_dir, "etc/od_20241104_sandag_airport_draftfinal.xlsx") # latest
variable_map_file = os.path.join(processed_dir, "revised_names.csv")
clean_survey_file = os.path.join(interim_dir, "survey_data_clean.csv")
output_csv_filename = os.path.join(processed_dir, "data_model_output.csv")
#summary_csv_filename = os.path.join(processed_dir, "data_model_output_summary.csv")

### Clean Data , Rename fields

In [5]:
in_df_complete = pd.read_excel(input_file, sheet_name = 0)
in_df_incomplete = pd.read_excel(input_file, sheet_name = 1)

in_df_complete['is_completed'] = 1
in_df_incomplete['is_completed'] = 0

in_df = pd.concat([in_df_complete, in_df_incomplete], ignore_index = True)
header_df = pd.read_csv(variable_map_file)[['ETC_name','WSP_name']]
header_dict = pd.Series(header_df.WSP_name.values,index=header_df.ETC_name).to_dict()
clean_df = in_df.rename(columns=header_dict).copy().drop(columns=["delete"])

In [17]:
print("Complete Records: ", in_df_complete.shape)
print("Incomplete Records: ", in_df_incomplete.shape)

Complete Records:  (3935, 359)
Incomplete Records:  (5062, 28)


In [6]:
clean_df.shape

(8997, 326)

In [11]:
len(clean_df['respondentid'].unique())

5063

In [14]:
#Remove the duplicate respondentids
clean_df.drop_duplicates('respondentid', keep = 'first', inplace = True)
clean_df.shape

(5063, 326)

In [16]:
clean_df[clean_df['is_completed']==1].shape

(3935, 326)

In [8]:
clean_df.head()

,respondentid,submit,date_completed,interview_location,interview_location_label,interview_location_other,inbound_or_outbound,inbound_or_outbound_label,marketsegment,marketsegment_label,...,REFUS_GENDER_OBSRVED,REFUS_RACETHN_OBSERV_1_,REFUS_RACETHN_OBSERV_2_,REFUS_RACETHN_OBSERV_3_,REFUS_RACETHN_OBSERV_4_,REFUS_RACETHN_OBSERV_5_,REFUS_RACETHN_OBSERV_6_,REFUS_RACETHN_OBSERV_7_,REFUS_RACETHN_OBSERV_8_,REFUS_RACETHN_OBSERV_Other_
0,5385,YES,2024-10-04,Term2,Terminal 2,NaN,IN,INBOUND,1.0,Air passenger,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5386,YES,2024-10-04,Term2,Terminal 2,NaN,IN,INBOUND,1.0,Air passenger,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5387,NO,2024-10-04,Term2,Terminal 2,NaN,IN,INBOUND,1.0,Air passenger,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5388,YES,2024-10-04,SDA_1_FLYER,San Diego Flyer/Old Town Shuttle,NaN,IN,INBOUND,1.0,Air passenger,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5389,YES,2024-10-04,Term2,Terminal 2,NaN,IN,INBOUND,1.0,Air passenger,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Commonly occuring invalid values

In [9]:
# Get the list of columns that contain '-oth-' as a value
columns_with_oth_value = [col for col in clean_df.columns if clean_df[col].eq('-oth-').any()]

print(columns_with_oth_value)

['interview_location', 'flight_purpose', 'employer', 'occupation', 'origin_activity_type', 'main_mode', 'car_available', 'reverse_mode_predicted', 'reverse_commute_mode']


In [10]:
columns_with_dash_value = [col for col in clean_df.columns if clean_df[col].eq('-').any()]

print(columns_with_dash_value)

['flight_number', 'origin_city', 'origin_state', 'origin_zip', 'destination_city', 'destination_state', 'destination_zip', 'transit_boarding_stop_name', 'transit_boarding_latitude', 'transit_boarding_longitude', 'transit_alighting_stop_name', 'transit_alighting_latitude', 'transit_alighting_longitude', 'home_location_city', 'home_location_state', 'home_location_zip', 'home_location_latitude', 'home_location_longitude']


### Making all modes consistent

In [11]:
clean_df['egress_mode_label'].value_counts()

egress_mode_label
Walk                                 25
Picked up by car by family/friend    11
Drive alone and park                  2
Uber/Lyft                             2
Other shared van (please specify)     1
Taxi                                  1
Name: count, dtype: int64

In [12]:
clean_df['other_airport_accessmode_label'].value_counts()

other_airport_accessmode_label
DROPPED OFF BY CAR BY FRIEND FAMILY               194
UBER LYFT                                          89
DROVE ALONE AND PARKED                             30
OTHER PUBLIC TRANSIT                               22
DROVE WITH OTHERS AND PARKED                       22
RENTAL CAR AND DROPPED IT OFF AT RENTAL AGENCY     17
TAXI                                               11
WHEELCHAIR OR OTHER MOBILITY DEVICE                 7
RENTAL CAR AND PARKED IT                            7
WALK                                                6
CAR SERVICE BLACK CAR LIMO EXECUTIVE CAR            4
NON ELECTRIC BIKESHARE                              2
RODE WITH OTHER TRAVELER AND PARKED                 2
HOTEL SHUTTLE VAN                                   2
PERSONAL NON ELECTRIC BICYCLE                       1
OTHER SHARED RIDE VAN SERVICE                       1
ELECTRIC BIKESHARE                                  1
EMPLOYEE SHUTTLE                                   

In [13]:
other_airport_accessmode_label_map = {
    'Walk': 'Walk',
    'Wheelchair or other mobility device': 'Wheelchair or other mobility device',
    'ELECTRIC BIKESHARE': 'Bicycle: electric bikeshare',
    'NON ELECTRIC BIKESHARE': 'Bicycle: non-electric bikeshare',
    'E SCOOTER SHARE': 'E-scooter: shared',
    'PERSONAL ELECTRIC BICYCLE': 'Bicycle: personal electric bicycle',
    'PERSONAL NON ELECTRIC BICYCLE': 'Bicycle: personal non-electric bicycle',
    'PERSONAL E SCOOTER': 'E-scooter: personal',
    'Taxi': 'Taxi',
    'UBER LYFT': 'Uber/Lyft',
    'CAR SERVICE BLACK CAR LIMO EXECUTIVE CAR': 'Car service/black car/limo/executive car',
    'DROPPED OFF BY CAR BY FRIEND FAMILY': 'Dropped off by car by family/friend',
    'Drove alone and parked': 'Drove alone and parked',
    'Drove with others and parked': 'Drove with others and parked',
    'RODE WITH OTHER TRAVELER AND PARKED': 'Rode with other traveler(s) and parked',
    'Other public transit': 'Other public transit',
    'Chartered tour bus': 'Chartered tour bus',
    'Employee shuttle': 'Employee shuttle',
    'RENTAL CAR AND DROPPED IT OFF AT RENTAL AGENCY': 'Rental car: Dropped off at rental agency',
    'RENTAL CAR AND PARKED IT': 'Rental car: parked rental car',
    'Hotel shuttle van': 'Hotel shuttle van',
    'OTHER SHARED RIDE VAN SERVICE': 'Other shared van (please specify)',
    'Other': 'Other',
    'Refused/No Answer': 'Refused/No Answer'
}
clean_df['other_airport_accessmode_label'] = clean_df['other_airport_accessmode_label'].map(other_airport_accessmode_label_map)

In [14]:
clean_df['other_airport_accessmode_label'].value_counts()

other_airport_accessmode_label
Dropped off by car by family/friend         194
Uber/Lyft                                    89
Rental car: Dropped off at rental agency     17
Rental car: parked rental car                 7
Car service/black car/limo/executive car      4
Rode with other traveler(s) and parked        2
Bicycle: non-electric bikeshare               2
Bicycle: personal non-electric bicycle        1
Other shared van (please specify)             1
Bicycle: electric bikeshare                   1
Name: count, dtype: int64

In [15]:
travel_mode_dict = {
    'Walk': 1,
    'Wheelchair or other mobility device': 2,
    'Bicycle: electric bikeshare': 3,
    'Bicycle: non-electric bikeshare': 4,
    'E-scooter: shared': 5,
    'Bicycle: personal electric bicycle': 6,
    'Bicycle: personal non-electric bicycle': 7,
    'E-scooter: personal': 8,
    'Taxi': 9,
    'Uber/Lyft': 10,
    'Car service/black car/limo/executive car': 11,
    'Dropped off by car by family/friend': 12,
    'Drove alone and parked': 13,
    'Drove with others and parked': 14,
    'MTS Route 992': 15,
    'Airport flyer shuttle': 16,
    'Chartered tour bus': 17,
    'Employee shuttle': 18,
    'Rental car: Dropped off at rental agency': 19,
    'Rental car: parked rental car': 20,
    'Hotel shuttle van': 21,
    'Other shared van (please specify)': 22,
    'Picked up by car by family/friend': 23,
    'Get in a parked vehicle and drive alone': 24,
    'Get in a parked vehicle and drive with others': 25,
    'Get in a parked vehicle and ride with other traveler(s)': 26,
    'Rental car: Picked up at rental agency': 27,
    'Rental car: get in a parked rental car': 28,
    'Rode with other traveler(s) and parked': 29,
    'Other public transit': 30,
    'Other': 98,
    'Refused/No Answer': 99,
    'None of the above': 98
}

### Modes to fix

In [16]:
mode_code_columns = ['main_transit_mode', 'main_mode', 'access_mode', 'egress_mode', 'reverse_mode', 'reverse_mode_predicted', 'other_airport_accessmode', 'reverse_commute_mode']
mode_label_columns = ['main_transit_mode_label', 'main_mode_label', 'access_mode_label', 'egress_mode_label', 'reverse_mode_label', 'reverse_mode_predicted_label', 'other_airport_accessmode_label', 'reverse_commute_mode_label']

In [17]:
#Remapping codes using label strings
for mode_code_col, mode_label_col in zip(mode_code_columns, mode_label_columns):
    # Apply the mapping for each pair of columns
    clean_df[mode_code_col] = clean_df[mode_label_col].map(travel_mode_dict)

In [18]:
clean_df['other_airport_accessmode_label'].value_counts()

other_airport_accessmode_label
Dropped off by car by family/friend         194
Uber/Lyft                                    89
Rental car: Dropped off at rental agency     17
Rental car: parked rental car                 7
Car service/black car/limo/executive car      4
Rode with other traveler(s) and parked        2
Bicycle: non-electric bikeshare               2
Bicycle: personal non-electric bicycle        1
Other shared van (please specify)             1
Bicycle: electric bikeshare                   1
Name: count, dtype: int64

In [19]:
clean_df['other_airport_accessmode'].value_counts()

other_airport_accessmode
12.0    194
10.0     89
19.0     17
20.0      7
11.0      4
29.0      2
4.0       2
7.0       1
22.0      1
3.0       1
Name: count, dtype: int64

In [20]:
clean_df['main_transit_mode'].value_counts()

main_transit_mode
98.0    3598
16.0     195
15.0     142
Name: count, dtype: int64

### Pre-processing of some fields

In [ ]:
clean_df['date_completed'] = pd.to_datetime(clean_df['date_completed'])
clean_df['record_type_synthetic'] = 0
clean_df.replace('-oth-', 98, inplace=True)
clean_df.replace('-', None, inplace = True )
clean_df['is_income_below_poverty'] = np.where(clean_df['is_income_below_poverty'] == 0, 2, clean_df['is_income_below_poverty'])
clean_df['stay_informed'] = np.where(clean_df['stay_informed'] == 0, 2, clean_df['stay_informed'])
#Maps
interview_location_map = {'Term1' : 1, 'Term2': 2, 'MTS_1_992': 3, 'SDA_1_FLYER': 4, 'ConracShuttle': 5, 'ParkingShuttle': 6, 'EmplParking': 7, '-oth-':98} 
inbound_outbound_map = {'IN':1, 'OUT':2}

#route_fields:
route_fields = ['to_airport_transit_route_1', 'to_airport_transit_route_2', 'to_airport_transit_route_3', 'to_airport_transit_route_4',
                'from_airport_transit_route_1', 'from_airport_transit_route_2', 'from_airport_transit_route_3', 'from_airport_transit_route_4']

#Replacement
clean_df['interview_location'] = clean_df['interview_location'].map(interview_location_map)
clean_df['inbound_or_outbound'] = clean_df['inbound_or_outbound'].map(inbound_outbound_map)
clean_df['main_mode'] = np.where(clean_df['main_transit_mode'].isin([15,16]), clean_df['main_transit_mode'], clean_df['main_mode'])

clean_df[route_fields] = clean_df[route_fields].replace(98, 'OTHER')
clean_df['nights_visited'] = clean_df['nights_visited'] - 1

clean_df['same_commute_mode'] = np.where(clean_df['same_commute_mode'] == 0, 2, clean_df['same_commute_mode'])
clean_df['resident_visitor_followup'] = np.where(clean_df['resident_visitor_followup'] == 0, 2, clean_df['resident_visitor_followup'])

#activity_type
clean_df['origin_activity_type'] = np.where(clean_df['inbound_or_outbound'] == e.InboundOutbound.OUTBOUND_FROM_AIRPORT, e.ActivityType.SAN_DIEGO_AIRPORT, clean_df['origin_activity_type'])
clean_df['destination_activity_type'] = np.where(clean_df['inbound_or_outbound'] == e.InboundOutbound.INBOUND_TO_AIRPORT, e.ActivityType.SAN_DIEGO_AIRPORT, clean_df['destination_activity_type'])

#For incomplete records:
clean_df['market_segment'] = clean_df['market_segment'].fillna(99)


C:\Users\USVV724227\AppData\Local\Temp\ipykernel_44332\2408829722.py:3: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  clean_df.replace('-oth-', 98, inplace=True)


In [22]:
clean_df['interview_location']

0       2.0
1       2.0
2       2.0
3       4.0
4       2.0
       ... 
8992    1.0
8993    1.0
8994    1.0
8995    1.0
8996    1.0
Name: interview_location, Length: 8997, dtype: float64

In [23]:
clean_df.to_csv(clean_survey_file, index = False)

In [24]:
clean_df.head()

,respondentid,submit,date_completed,interview_location,interview_location_label,interview_location_other,inbound_or_outbound,inbound_or_outbound_label,marketsegment,marketsegment_label,...,REFUS_RACETHN_OBSERV_1_,REFUS_RACETHN_OBSERV_2_,REFUS_RACETHN_OBSERV_3_,REFUS_RACETHN_OBSERV_4_,REFUS_RACETHN_OBSERV_5_,REFUS_RACETHN_OBSERV_6_,REFUS_RACETHN_OBSERV_7_,REFUS_RACETHN_OBSERV_8_,REFUS_RACETHN_OBSERV_Other_,record_type_synthetic
0,5385,YES,2024-10-04,2.0,Terminal 2,NaN,1.0,INBOUND,1.0,Air passenger,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
1,5386,YES,2024-10-04,2.0,Terminal 2,NaN,1.0,INBOUND,1.0,Air passenger,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
2,5387,NO,2024-10-04,2.0,Terminal 2,NaN,1.0,INBOUND,1.0,Air passenger,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,5388,YES,2024-10-04,4.0,San Diego Flyer/Old Town Shuttle,NaN,1.0,INBOUND,1.0,Air passenger,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,5389,YES,2024-10-04,2.0,Terminal 2,NaN,1.0,INBOUND,1.0,Air passenger,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


### Select Variables to verify for the survey

In [25]:
respondent_variables = [field_name for field_name, field_info in Respondent.__fields__.items()]


trip_variables = [field_name for field_name, field_info in Trip.__fields__.items()]
trip_variables.append('respondentid')

employee_variables = [field_name for field_name, field_info in Employee.__fields__.items()]
employee_variables.remove('trip')

air_passenger_variables = [field_name for field_name, field_info in AirPassenger.__fields__.items()]
air_passenger_variables.remove('trip')

variables_to_verify = list(set(air_passenger_variables + respondent_variables + trip_variables + employee_variables))
variables_to_verify.remove('trip')
variables_to_verify.remove('valid_record')
variables_to_verify.remove('validation_error')
variables_to_verify.remove('validation_severity')


working_df = clean_df.copy()
working_df = working_df[variables_to_verify].copy()
working_df = working_df.loc[working_df['marketsegment'].notna()].copy()
working_df.head()

,sdia_accessmode_split_walk,flight_number,origin_state,destination_state,age,taxi_fhv_wait,shift_end_time,passenger_type,alt_commute_mode_airport_flyer_shuttle,country_of_residence,...,number_workers,general_modes_used_visitor_other_public_bus,same_commute_mode_other,to_airport_transit_route_1,transit_boarding_latitude,sdia_accessmode_split_rental_car_parked,alt_commute_mode_e_scooter_shared,reasons_no_transit_ride_too_long,general_use_transit_visitor_home,sdia_accessmode_decision
0,NaN,8763,CA,CA,9.0,5m8n,NaN,2.0,NaN,35.0,...,3.0,No,NaN,NaN,None,NaN,NaN,NaN,0.0,NaN
1,No,Ua 8496,CA,CA,3.0,5 minutes,NaN,2.0,NaN,NaN,...,2.0,NaN,NaN,NaN,None,No,NaN,NaN,NaN,2.0
2,NaN,Ua8496,CA,NaN,4.0,NaN,NaN,2.0,NaN,11.0,...,0.0,No,NaN,MTS_1_1,32.731527,NaN,NaN,NaN,1.0,NaN
3,NaN,Don't know,CA,CA,9.0,NaN,NaN,2.0,NaN,NaN,...,2.0,NaN,NaN,NCT_2_398,32.755309,NaN,NaN,NaN,NaN,NaN
4,NaN,Ac526,CA,CA,10.0,Refused/No Answer,NaN,2.0,NaN,NaN,...,3.0,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN


In [26]:
working_df.shape

(8952, 245)

### Serialize the data

In [27]:
trips_df = working_df[trip_variables].copy()
persons_df = working_df[list[set(employee_variables + respondent_variables + air_passenger_variables)]].copy()

In [28]:
# combined
respondent_list = add_list_objects(
        trips_df.to_dict(orient="records"),  #child list
        "respondentid", # child key
        persons_df.to_dict(orient="records"), # parent list
        "respondentid", # parent key
        "trip", # parent var
    )

In [29]:
len(respondent_list)

8952

In [30]:
employee_list = []
air_passenger_list = []
other_list = []
failed_records = []

for respondent in respondent_list:
    market_segment = respondent["marketsegment"]
    try:
        if market_segment == e.Type.EMPLOYEE:
            ev = Employee(** respondent)
            employee_list.append(ev)
        elif market_segment == e.Type.PASSENGER:
             av = AirPassenger(** respondent)
             air_passenger_list.append(av)
        else:
            rv = Respondent(** respondent)
            other_list.append(rv)
    except ValidationError as err:
            respondent['error_flag'] = 'failed'
            respondent['error_message'] = str(err)
            failed_records.append(respondent) 


failed_df = pd.DataFrame(failed_records)
failed_df.head()

""


In [31]:
failed_df.shape

(0, 0)

In [32]:
#failed_df['error_message'].unique()

In [33]:
#failed_df.to_csv('../data/processed/failed_records.csv', index = False)

In [34]:
len(failed_df)

0

### Make Data

In [35]:
employee_df = pd.DataFrame([Employee.model_dump() for Employee in employee_list])       

In [36]:
passenger_df = pd.DataFrame([AirPassenger.model_dump() for AirPassenger in air_passenger_list])

c:\Users\USVV724227\AppData\Local\Programs\Python\Python312\Lib\site-packages\pydantic\main.py:364: UserWarning: Pydantic serializer warnings:
  Expected `enum` but got `Terminal` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [37]:
other_df = pd.DataFrame([Respondent.model_dump() for Respondent in other_list])
# other_df = add_enum_label_columns(other_df, Respondent)

In [45]:
len(other_list)

16

In [39]:
trip_list = []
id_list = []
for record in employee_list + air_passenger_list + other_list:
    trip_list.append(record.trip)
    id_list.append(record.respondentid)

trip_df = pd.DataFrame([Trip.model_dump() for Trip in trip_list])
id_df = pd.DataFrame(id_list, columns=["respondentid"])

trip_df = pd.concat([id_df, trip_df], axis=1)
trip_df = add_enum_label_columns(trip_df,Trip)


In [40]:
output_df = pd.concat([employee_df, passenger_df, other_df], axis=0).reset_index(drop=True).drop(columns=["trip"])
output_df = pd.merge(output_df, trip_df, on="respondentid", how="left")

C:\Users\USVV724227\AppData\Local\Temp\ipykernel_44332\2189731026.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  output_df = pd.concat([employee_df, passenger_df, other_df], axis=0).reset_index(drop=True).drop(columns=["trip"])


In [41]:
output_df.shape

(16820, 272)

In [ ]:
output_df = add_synthetic_records(output_df)

In [42]:
output_df =  add_enum_label_columns(output_df, Respondent)
output_df =  add_enum_label_columns(output_df, AirPassenger)
output_df =  add_enum_label_columns(output_df, Trip)
output_df =  add_enum_label_columns(output_df, Employee)

In [43]:
output_df['destination_activity_type_label'].value_counts()

destination_activity_type_label
SAN_DIEGO_AIRPORT    16222
nan                    598
Name: count, dtype: int64

In [44]:
reference_columns = clean_df.columns.tolist()
#reference_columns.remove('home_location_address')

# Get the middle columns (those not in reference_columns) from output_df
remaining_columns = [col for col in output_df.columns if col not in reference_columns]

# Sort the remaining columns alphabetically
sorted_remaining_columns = sorted(remaining_columns)

# Create the new column order using reference_columns and sorted remaining columns
new_column_order = reference_columns + sorted_remaining_columns

# Reorder the DataFrame
output_df = output_df[new_column_order]

# Display the updated DataFrame
output_df.head()

KeyError: "['Date started', 'HAVE_5_MIN_FOR_SURVE_Code_', 'HAVE_5_MIN_FOR_SURVE', 'REFUS_AGE_OBSERVED_Code_', 'REFUS_AGE_OBSERVED', 'REFUS_GENDER_OBSRVED_Code_', 'REFUS_GENDER_OBSRVED', 'REFUS_RACETHN_OBSERV_1_', 'REFUS_RACETHN_OBSERV_2_', 'REFUS_RACETHN_OBSERV_3_', 'REFUS_RACETHN_OBSERV_4_', 'REFUS_RACETHN_OBSERV_5_', 'REFUS_RACETHN_OBSERV_6_', 'REFUS_RACETHN_OBSERV_7_', 'REFUS_RACETHN_OBSERV_8_', 'REFUS_RACETHN_OBSERV_Other_'] not in index"

In [ ]:
output_df.to_csv(output_csv_filename, index = False)